# Background

## Introduction

### Problem

Earth scientists work with huge, 4+ dimensional datasets (3
spatial dimensions, 1 time dimension, and sometimes even more).

- Example: Air temperature (longitude, latitude, height, time)
- Example: Surface pressure from "ensemble" of model simulations
  (longitude, latitude, time, ensemble member)
- Example: Satellite-retrieved radiation (projection x-coordinate,
  projection y-coordinate, wavenumber)
    
### Question

What's the best way to **store** these 4+ dimensional datasets?

- Spreadsheets? Not enough dimensions.
- Matrices? No way to annotate "rows", "columns", etc.

### Answer

The [NetCDF](https://en.wikipedia.org/wiki/NetCDF) format
(Network Common Data Form) developed by
[UCAR/Unidata](https://www.unidata.ucar.edu/software/netcdf/) (right
down the road!).

- Description: Annotated N-dimensional matrices (arrays)
- File extension: `.nc`

There are other similar data formats
([HDF](https://en.wikipedia.org/wiki/Hierarchical_Data_Format),
[GRIB](https://en.wikipedia.org/wiki/GRIB)), and some software can work
seamlessly with different formats... but **NetCDF is your new best friend**.

## Architecture

### Basics

NetCDF files have the following features:

- Named dimensions, describing the dimensions on the arrays contained in the NetCDF file.
- Named variables, i.e. the data arrays, each with their own dimensions and attributes.
- "Coordinate variables", named variables whose name matches a dimension name. **Coordinate variables are optional.**
- Global attributes, describing the contents of the NetCDF file. **Attributes are optional.**

Here are some variable attributes [associated with the CF standard](http://cfconventions.org/cf-conventions/v1.6.0/cf-conventions.html#variables) you should know about:

- `long_name`: A descriptive, human-readable name (as opposed to the "short" variable name), e.g. the variable `'t'` might have `long_name = 'air temperature'`.
- `standard_name`: An unambiguous **unique** variable name, laid out by the CF standard. Used by software to interpret stuff.
- `units`: A string indicating the variable units.
- `_FillValue`: The value used to indicate missing data. Applied in xarray as NaNs and in netcdf4 as masked values.
- `add_offset`, `scale_factor`: Constants used to help compress files that must be applied to variable data using `scaled = add_offset + scale_factor * unscaled`.


Example: A simple file containing [HadCRUT](https://crudata.uea.ac.uk/cru/data/temperature/) land and sea surface temperature data (throughout this notebook, we trigger command-line commands with the `%%bash` magic).

In [27]:
%%bash
ncdump -h data/sst.nc

netcdf sst {
dimensions:
	latitude = 36 ;
	longitude = 72 ;
	field_status_string_length = 1 ;
	time = UNLIMITED ; // (2010 currently)
variables:
	float latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:point_spacing = "even" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	float longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "longitude" ;
		longitude:point_spacing = "even" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:units = "days since 1850-1-1 00:00:00" ;
		time:calendar = "gregorian" ;
		time:start_year = 1850s ;
		time:end_year = 2017s ;
		time:start_month = 1s ;
		time:end_month = 6s ;
		time:axis = "T" ;
	float temperature_anomaly(time, latitude, longitude) ;
		temperature_anomaly:long_name = "near_surface_temperature_anomaly" ;
		temperature_anomaly:units = "K" 

### Versions

NetCDF files also have [different "version
numbers"](https://www.unidata.ucar.edu/software/netcdf/docs/faq.html#How-many-netCDF-formats-are-there-and-what-are-the-differences-among-them):


-   NetCDF3 (version 3) [retired in
    2008](https://www.unidata.ucar.edu/software/netcdf/docs/RELEASE_NOTES.html#autotoc_md60).
    Unidata is [currently on version
    4](https://www.unidata.ucar.edu/software/netcdf/docs/RELEASE_NOTES.html#autotoc_md0).
-   However version 3 still widespread. Scientists are slow to change
    their ways... too many other things to worry about.
-   Some things in NetCDF4 are impossible in NetCDF3 (e.g., multiple unlimited
    dimensions), and NetCDF4 is a bit smaller, so use whenever possible.
-   Weird read/write bugs can arise due to version
    incompatibilities.

NetCDF3 is still everywhere, so you may need to use it.

In [25]:
%%bash
ncdump -k data/sst.nc

netCDF-4


In [26]:
%%bash
ncdump -k data/landfracs.nc

classic


### Software

There's lots of software for working with NetCDF (HDF, GRIB) files. In general, the software fits into one two categories:

**Command-line** tools:

* Can be used in the UNIX command line or written in a shell script.
* Generally require saving NetCDF files at each step.
* Can be concise, but have more limited applications.
* Example usage: Getting monthly averages or ensemble averages, converting between horizontal and vertical grids.

Note "shell scripts" are just text files containing commands that can be typed into the UNIX command line. Like a python script, they can be run repeatedly. Shell scripts generally have the extension `.sh` and can be run with `bash script.sh` or `./script.sh` (the latter requires that the script is given execute permissions -- for example, with `chmod 755 script.sh`).

**Python** tools.

* Can be used with interactive tools like `ipython`, `jupyter`, and `spyder`.
* Generally don't require saving NetCDF files at every step.
* Can be verbose, but have more broad applications.
* Example usage: Calculating physical quantities from NetCDF variables, creating plots from NetCDF variables.

Command-line tools and python tools can both be really useful! But you will probably find yourself using python tools the most.

I also have a work-in-progress Github page called [atmos-benchmarks](https://github.com/lukelbd/atmos-benchmarks) that compares the performance of different tools on a few different platforms and for a few simple tasks. You will notice some obvious patterns: NCO and NCL tend to be slowest, CDO tends to be fastest,  and xarray paired with dask can be extremely fast (see below for what all this means). However, take these with a grain of salt! In general, on a day-to-day basis, the "best" tool to use is the one that can do the job with the least amount of coding.

# Command-line tools

First, the command-line tools. There are just two popular tools you should know about:

- [NCO](http://nco.sourceforge.net/nco.html) (NetCDF Operators).
- [CDO](https://code.mpimet.mpg.de/projects/cdo/wiki/Cdo#Documentation) (Climate Data Operators).

## NCO

- Developed by Unidata, the makers of the NetCDF format.
- Extremely versatile but a bit clunky to learn.
- Tend to be slower.


These consist of a suite of separate commands:

- ncap2: netCDF Arithmetic Processor
- ncatted: netCDF ATTribute EDitor (examples)
- ncbo: netCDF Binary Operator (addition, multiplication...)
- ncclimo: netCDF CLIMatOlogy Generator
- nces: netCDF Ensemble Statistics
- ncecat: netCDF Ensemble conCATenator
- ncflint: netCDF FiLe INTerpolator
- ncks: netCDF Kitchen Sink
- ncpdq: netCDF Permute Dimensions Quickly, Pack Data Quietly
- ncra: netCDF Record Averager
- ncrcat: netCDF Record conCATenator
- ncremap: netCDF REMAPer
- ncrename: netCDF RENAMEer
- ncwa: netCDF Weighted Averager

I recommend using `nco` for straightforward operations like `ncatted` and `ncrename` and `cdo` for more complicated operations.

### Basics

I really don't use NCO very much... so I need to do a little research before expanding this section.

### Example

The following example reads XYZT temperature and wind data, then saves YZT "eddy heat flux".

In [20]:
%%bash
ncdump -h data/atmosphere.nc

netcdf atmosphere {
dimensions:
	time = UNLIMITED ; // (200 currently)
	plev = 60 ;
	lat = 6 ;
	lon = 12 ;
variables:
	float time(time) ;
		time:long_name = "time" ;
		time:calendar = "360_day" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
	float plev(plev) ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:axis = "Z" ;
		plev:standard_name = "pressure level" ;
	float lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
		lat:standard_name = "latitude" ;
	float lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degE" ;
		lon:axis = "X" ;
		lon:standard_name = "longitude" ;
	double u(time, plev, lat, lon) ;
		u:long_name = "zonal wind" ;
		u:units = "m/s" ;
	double v(time, plev, lat, lon) ;
		v:long_name = "meridional wind" ;
		v:units = "m/s" ;
	double t(time, plev, lat, lon) ;
		t:long_name = "temperature" ;
		t:units = "K" ;
}


In [25]:
%%bash
ncap2 -O -v -s '
  emf = ((u - u.avg($lon)) * (v - v.avg($lon))).avg($lon);
  ehf = ((t - t.avg($lon)) * (v - v.avg($lon))).avg($lon);
  ' data/atmosphere.nc data/atmosphere_nco.nc
ncatted data/atmosphere_nco.nc \
    -a long_name,ehf,m,c,'eddy heat flux' \
    -a units,ehf,m,c,'K m / s' \
    -a long_name,emf,m,c,'eddy momentum flux' \
    -a units,emf,m,c,'m^2 / s'

In [26]:
%%bash
ncdump -h data/atmosphere_nco.nc

netcdf atmosphere_nco {
dimensions:
	time = UNLIMITED ; // (200 currently)
	plev = 60 ;
	lat = 6 ;
	lon = 12 ;
variables:
	double emf(time, plev, lat) ;
		emf:long_name = "eddy momentum flux" ;
		emf:units = "m^2 / s" ;
	double ehf(time, plev, lat) ;
		ehf:long_name = "eddy heat flux" ;
		ehf:units = "K m / s" ;
	float time(time) ;
		time:long_name = "time" ;
		time:calendar = "360_day" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
	float plev(plev) ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:axis = "Z" ;
		plev:standard_name = "pressure level" ;
	float lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
		lat:standard_name = "latitude" ;
	float lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degE" ;
		lon:axis = "X" ;
		lon:standard_name = "longitude" ;

// global attributes:
		:history = "Sat Jun 12 13:42:52 2021: ncatted data/atmosphere_nco.nc -a long_name,ehf,

## CDO


- Developed by the Max Planck Institute for Meteorology.
- Sleeker and more "modern" NetCDf tool.
- Consists of a single command-line command `cdo` but with [hundreds of subcommands](https://code.mpimet.mpg.de/projects/cdo/wiki/Tutorial#Basic-Usage).
- Inclues a game-changer "[operator chaining](https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-120001.2.6)" feature.
- Powered by C++ with built-in parallelization, so this tends to be faster.
- Includes an optional [python API](https://code.mpimet.mpg.de/projects/cdo/wiki/Cdo%7Brbpy%7D) for calling CDO commands within python code.
- The only major caveat: Your data must fit a rigid format, with 2 horizontal dimensions, an optional vertical dimension, and an optional time dimension.
- CDO must be able to infer these dimensions based on the attributes (using the [CF-standards for coordinates](http://cfconventions.org/cf-conventions/cf-conventions.html#coordinate-types)).


CDO has useful file-inspection utilities you should know about:

- `griddes` displays the horizontal "grids" interpreted by CDO for commands like `fldmean`.
- `zaxisdes` displays the vertical "axes" interpreted by CDO for commands like `ml2pl`.
- `infon` and `sinfon` summarize data variable contents in a nice table.

I recommend CDO for complex operations, whenever the `input_file.nc --> output_file.nc` workflow is appropriate, and whenever your datasets are characterized by "horizontal dimensions", "vertical dimensions", and "time dimensions".

Here's a quick survey of CDO's subcommands, with examples.

### Basics

CDO can regrid from one arbitrary grid (e.g. rotated pole, hexagonal cells) to another (you can choose from a suite of algorithms). The regridding commands include `remapbil` for bilinear interpolation, `remapnn` for nearest-neighbor interpolation, and `remapcon` for area-conserving weighted averaging.

The following example interpolates "land fractions" (the fractions of grid cell areas containing land as opposed to ocean) onto a low-resolution sea-surface temperature dataset using an area-conserving remapping scheme `remapcon`.

In [51]:
%%bash
cdo griddes data/sst.nc
cdo griddes data/landfracs.nc

#
# gridID 1
#
gridtype  = lonlat
gridsize  = 2592
datatype  = float
xsize     = 72
ysize     = 36
xname     = longitude
xlongname = "longitude"
xunits    = "degrees_east"
yname     = latitude
ylongname = "latitude"
yunits    = "degrees_north"
xfirst    = -177.5
xinc      = 5
yfirst    = -87.5
yinc      = 5
#
# gridID 1
#
gridtype  = lonlat
gridsize  = 1036800
datatype  = float
xsize     = 1440
ysize     = 720
xname     = lon
xlongname = "longitude"
xunits    = "degrees_east"
yname     = lat
ylongname = "latitude"
yunits    = "degrees_north"
xfirst    = 0.125
xinc      = 0.25
yfirst    = 89.875
yinc      = -0.25


Warning (cdfCheckVars): Unsupported data type (char/string), skipped variable field_status!
cdo    griddes: Processed 1 variable [0.01s 9804KB].
cdo    griddes: Processed 1 variable [0.00s 7456KB].


In [55]:
%%bash
cdo remapcon,data/sst.nc data/landfracs.nc data/sst_landfracs.nc

Warning (cdfCheckVars): Unsupported data type (char/string), skipped variable field_status!
cdo    remapcon: YAC first order conservative weights from lonlat (1440x720) to lonlat (72x36) grid
cdo    remapcon: Processed 1036800 values from 1 variable over 1 timestep [2.17s 90MB].


In [57]:
%%bash
cdo griddes data/sst_landfracs.nc

#
# gridID 1
#
gridtype  = lonlat
gridsize  = 2592
datatype  = float
xsize     = 72
ysize     = 36
xname     = longitude
xlongname = "longitude"
xunits    = "degrees_east"
yname     = latitude
ylongname = "latitude"
yunits    = "degrees_north"
xfirst    = -177.5
xinc      = 5
yfirst    = -87.5
yinc      = 5


cdo    griddes: Processed 1 variable [0.01s 7316KB].


CDO can also interpolate to different vertical levels and change the vertical dimension type.

The following example interpolates input data with variable pressure levels to 100hPa interval levels. Many CDO commands accept either comma-separated lists (e.g., `intlevel,1000,900,800`) or slash-separated range indicators (e.g., `intlevel,1000,100,-100`).

In [48]:
%%bash
cdo intlevel,1000/100/-100 data/atmosphere.nc data/atmosphere_100.nc
ncdump -v plev data/atmosphere_100.nc

netcdf atmosphere_100 {
dimensions:
	time = UNLIMITED ; // (200 currently)
	lon = 12 ;
	lat = 6 ;
	plev = 10 ;
variables:
	float time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:calendar = "360_day" ;
		time:axis = "T" ;
	float lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degE" ;
		lon:axis = "X" ;
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
	float plev(plev) ;
		plev:standard_name = "air_pressure" ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:positive = "down" ;
		plev:axis = "Z" ;
	double u(time, plev, lat, lon) ;
		u:long_name = "zonal wind" ;
		u:units = "m/s" ;
	double v(time, plev, lat, lon) ;
		v:long_name = "meridional wind" ;
		v:units = "m/s" ;
	double t(time, plev, lat, lon) ;
		t:long_name = "temperature" ;
		t:units = "K" ;

// global attribut

cdo    intlevel: Processed 2592000 values from 3 variables over 200 timesteps [0.14s 13MB].


CDO can also handle daily, monthly, seasonal, and annual statistics and spatial operations.

There is an [enormous number](https://code.mpimet.mpg.de/projects/cdo/embedded/index.html#x1-3630002.8) of statistical CDO subcommands. They are all extremely useful and extremely powerful. The below example demonstrates just a few of these commands for the sea surface temperature data.

In [97]:
%%bash
cdo ymonmean data/sst.nc data/sst_monthly.nc  # monthly means
cdo zonmean data/sst.nc data/sst_zonal.nc  # longitude average
cdo fldmean data/sst.nc data/sst_global.nc  # global average

Warning (cdfCheckVars): Unsupported data type (char/string), skipped variable field_status!
cdo    ymonmean: Processed 5209920 values from 1 variable over 2010 timesteps [0.29s 33MB].
Warning (cdfCheckVars): Unsupported data type (char/string), skipped variable field_status!
cdo    zonmean: Processed 5209920 values from 1 variable over 2010 timesteps [0.43s 35MB].
Warning (cdfCheckVars): Unsupported data type (char/string), skipped variable field_status!
cdo    fldmean: Processed 5209920 values from 1 variable over 2010 timesteps [0.40s 34MB].


In [100]:
%%bash
ncdump -h data/sst_monthly.nc  # note the length-12 time axis, 1 for each month
ncdump -h data/sst_zonal.nc  # note the length-1 longitude axis, indicating a longitude average
ncdump -h data/sst_global.nc  # note the length-1 latitude and longitude axes, indicating a global average

netcdf sst_monthly {
dimensions:
	time = UNLIMITED ; // (12 currently)
	bnds = 2 ;
	longitude = 72 ;
	latitude = 36 ;
variables:
	float time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1850-1-1 00:00:00" ;
		time:calendar = "gregorian" ;
		time:axis = "T" ;
	double time_bnds(time, bnds) ;
	float longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "longitude" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	float temperature_anomaly(time, latitude, longitude) ;
		temperature_anomaly:long_name = "near_surface_temperature_anomaly" ;
		temperature_anomaly:units = "K" ;
		temperature_anomaly:_FillValue = -1.e+30f ;
		temperature_anomaly:missing_value = -1.e+30f ;
		temperature_anomaly:reference_pe

CDO can even handle fairly advanced objective-analysis tasks, like calculating [empirical orthogonal functions](https://en.wikipedia.org/wiki/Empirical_orthogonal_functions) (EOFs) and performaing [linear least-squares regressions](https://en.wikipedia.org/wiki/Linear_least_squares) (LLS). This is generally quite complicated when using other tools, but CDO makes it easy.

The below example calculates LLS trendlines for the pointwise and global-mean sea surface temperature data between 1980 and 2010.

In [107]:
%%bash
cdo regres -mulc,1 -seldate,1980-01-01T00:00:00,2009-12-31T23:59:59 data/sst.nc data/sst_regress.nc  # trend in K/year
cdo regres -mulc,1 -fldmean -seldate,1980-01-01T00:00:00,2009-12-31T23:59:59 data/sst.nc data/sst_regress_global.nc  # trend in K/year

cdo(1) mulc: Process started
cdo(2) seldate: Process started
Warning (cdfCheckVars): Unsupported data type (char/string), skipped variable field_status!
cdo(2) seldate: Processed 933120 values from 1 variable over 1921 timesteps.
cdo(1) mulc: Processed 933120 values from 1 variable over 360 timesteps.
cdo    regres: Processed 933120 values from 1 variable over 360 timesteps [0.12s 17MB].
cdo(1) mulc: Process started
cdo(2) fldmean: Process started
cdo(3) seldate: Process started
Warning (cdfCheckVars): Unsupported data type (char/string), skipped variable field_status!
cdo(3) seldate: Processed 933120 values from 1 variable over 1921 timesteps.
cdo(2) fldmean: Processed 933120 values from 1 variable over 360 timesteps.
cdo(1) mulc: Processed 360 values from 1 variable over 360 timesteps.
cdo    regres: Processed 360 values from 1 variable over 360 timesteps [0.11s 17MB].


In [109]:
%%bash
ncdump -h data/sst_regress.nc
ncdump -h data/sst_regress_global.nc

netcdf sst_regress {
dimensions:
	time = UNLIMITED ; // (1 currently)
	longitude = 72 ;
	latitude = 36 ;
variables:
	float time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:units = "days since 1850-1-1 00:00:00" ;
		time:calendar = "gregorian" ;
		time:axis = "T" ;
	float longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "longitude" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	float temperature_anomaly(time, latitude, longitude) ;
		temperature_anomaly:long_name = "near_surface_temperature_anomaly" ;
		temperature_anomaly:units = "K" ;
		temperature_anomaly:_FillValue = -1.e+30f ;
		temperature_anomaly:missing_value = -1.e+30f ;
		temperature_anomaly:reference_period = 1961s, 1990s ;

// global attributes:
		:CDI = "Climate Data Interfa

### Example

The following example reads XYZT temperature and wind data, then saves YZT "eddy heat flux" and "eddy momentum flux".

In [27]:
%%bash
ncdump -h data/atmosphere.nc

netcdf atmosphere {
dimensions:
	time = UNLIMITED ; // (200 currently)
	plev = 60 ;
	lat = 6 ;
	lon = 12 ;
variables:
	float time(time) ;
		time:long_name = "time" ;
		time:calendar = "360_day" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
	float plev(plev) ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:axis = "Z" ;
		plev:standard_name = "pressure level" ;
	float lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
		lat:standard_name = "latitude" ;
	float lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degE" ;
		lon:axis = "X" ;
		lon:standard_name = "longitude" ;
	double u(time, plev, lat, lon) ;
		u:long_name = "zonal wind" ;
		u:units = "m/s" ;
	double v(time, plev, lat, lon) ;
		v:long_name = "meridional wind" ;
		v:units = "m/s" ;
	double t(time, plev, lat, lon) ;
		t:long_name = "temperature" ;
		t:units = "K" ;
}


In [84]:
%%bash
cdo -O \
  -setattribute,ehf@long_name='eddy heat flux',ehf@units='K m / s',emf@long_name='eddy momentum flux',emf@units='m^2 / s' \
  -expr,'
      ehf = zonmean((t - zonmean(t)) * (v - zonmean(v)));
      emf = zonmean((u - zonmean(u)) * (v - zonmean(v)));
  ' data/atmosphere.nc data/atmosphere_cdo.nc

cdo(1) expr: Process started
cdo(1) expr: Processed 2592000 values from 3 variables over 200 timesteps.
cdo    setattribute: Processed 144000 values from 2 variables over 200 timesteps [0.39s 16MB].


In [85]:
%%bash
ncdump -h data/atmosphere_cdo.nc

netcdf atmosphere_cdo {
dimensions:
	time = UNLIMITED ; // (200 currently)
	lon = 1 ;
	lat = 6 ;
	plev = 60 ;
variables:
	float time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:calendar = "360_day" ;
		time:axis = "T" ;
	double lon(lon) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "longitude" ;
		lon:units = "degrees_east" ;
		lon:axis = "X" ;
	double lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "latitude" ;
		lat:units = "degrees_north" ;
		lat:axis = "Y" ;
	float plev(plev) ;
		plev:standard_name = "pressure level" ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:positive = "down" ;
		plev:axis = "Z" ;
	float ehf(time, plev, lat, lon) ;
		ehf:_FillValue = -9.e+33f ;
		ehf:missing_value = -9.e+33f ;
		ehf:long_name = "eddy heat flux" ;
		ehf:units = "K m / s" ;
	float emf(time, plev, lat, lon) ;
		emf:_FillValue = -9.e+33f ;
		emf:missing_value = -9.e+33f ;

# Python tools

Next, the python tools:

- [netCDF4](https://unidata.github.io/netcdf4-python/)
- [xarray](http://xarray.pydata.org/en/stable/)
- [iris "cubes"](https://scitools.org.uk/iris/docs/v1.13.0/userguide/loading_iris_cubes.html) -- this tool is older, less widely used, falling out of favor (xarray was built as an [improvement on "cubes"](http://xarray.pydata.org/en/stable/getting-started-guide/faq.html#what-other-netcdf-related-python-libraries-should-i-know-about)).  
- [cf-python](https://github.com/NCAS-CMS/cf-python) -- this tool is not widely used.

We will focus on the two most widely used tools, netCDF4 and xarray.

## netCDF4

- The more "low-level" tool (requires more lines of code).
- Generally the fastest tool (unless you are using dask on a supercomputer -- see below).
- Used *internally* in the xarray source code.
- Includes very basic `Dataset` and `Variable` classes representing NetCDF files and NetCDF variables (respectively).
- The classes are used mainly to load variables into `numpy` arrays -- they do **cannot** interpret CF standards or apply `add_offset` or `scale_factor`.
- To create new NetCDF files, you must use an arcane and counter-intuitive API. Lots of bookkeeping.
- **Warning**: `netCDF4` works with both NetCDF versions 3 and 4!

I recommend using netCDF4 only if you have a very specific reason to use it over xarray (e.g., running calculations on many many small files on a laptop or individual server).

### Basics

The `Dataset` looks like this.

In [63]:
import netCDF4 as nc4
data = nc4.Dataset('data/atmosphere.nc')
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): time(200), plev(60), lat(6), lon(12)
    variables(dimensions): float32 time(time), float32 plev(plev), float32 lat(lat), float32 lon(lon), float64 u(time,plev,lat,lon), float64 v(time,plev,lat,lon), float64 t(time,plev,lat,lon)
    groups: 

A `Variable` is accessed by indexing the `Dataset` like a dictionary.

In [66]:
data['t']

<class 'netCDF4._netCDF4.Variable'>
float64 t(time, plev, lat, lon)
    long_name: temperature
    units: K
unlimited dimensions: time
current shape = (200, 60, 6, 12)
filling on, default _FillValue of 9.969209968386869e+36 used

To extract a masked numpy array from the `Variable`, use additional array-style indexing.

In [74]:
data['t'][:]  # select all data
data['t'][0, :, :, :]  # select the first timestep
data['t'][0, ...]  # numpy's shorthand for the above
type(data['t'][:])

numpy.ma.core.MaskedArray

### Example

The following example reads XYZT temperature and wind data, then saves YZT "eddy heat flux".

In [11]:
%%bash
ncdump -h data/atmosphere.nc

netcdf atmosphere {
dimensions:
	time = UNLIMITED ; // (200 currently)
	plev = 60 ;
	lat = 6 ;
	lon = 12 ;
variables:
	float time(time) ;
		time:long_name = "time" ;
		time:calendar = "360_day" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
	float plev(plev) ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:axis = "Z" ;
		plev:standard_name = "pressure level" ;
	float lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
		lat:standard_name = "latitude" ;
	float lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degE" ;
		lon:axis = "X" ;
		lon:standard_name = "longitude" ;
	double u(time, plev, lat, lon) ;
		u:long_name = "zonal wind" ;
		u:units = "m/s" ;
	double v(time, plev, lat, lon) ;
		v:long_name = "meridional wind" ;
		v:units = "m/s" ;
	double t(time, plev, lat, lon) ;
		t:long_name = "temperature" ;
		t:units = "K" ;
}


In [96]:
import os
import sys
import netCDF4 as nc4

# Open file
getattrs = lambda obj: {key: val for key, val in obj.__dict__.items() if key[:1] != '_'}
with nc4.Dataset('data/atmosphere.nc', 'r') as f:
    # Get dimensions
    time_var = f['time']
    plev_var = f['plev']
    lon_var = f['lon']
    lat_var = f['lat']
    time = time_var[:]
    plev = plev_var[:]
    lon = lon_var[:]
    lat = lat_var[:]

    # Get numpy arrays
    u_var = f['u']
    v_var = f['v']
    t_var = f['t']
    u = u_var[:]
    v = v_var[:]
    t = t_var[:]

    # Get attributes
    time_attrs = {attr: time_var.getncattr(attr) for attr in time_var.ncattrs()}
    plev_attrs = {attr: plev_var.getncattr(attr) for attr in plev_var.ncattrs()}
    lat_attrs = {attr: lat_var.getncattr(attr) for attr in lat_var.ncattrs()}

# Calculations
emf = ((u - u.mean(axis=-1, keepdims=True)) * (v - v.mean(axis=-1, keepdims=True))).mean(axis=-1)
ehf = ((t - t.mean(axis=-1, keepdims=True)) * (v - v.mean(axis=-1, keepdims=True))).mean(axis=-1)

# Save file
with nc4.Dataset('data/atmosphere_netcdf4.nc', 'w') as f:
    # Make dimensions
    # NOTE: This is very similar syntax to low-level C and Fortran libs
    f.createDimension('time', None)
    f.createDimension('plev', plev.size)
    f.createDimension('lat', lat.size)

    # Make coordinate Variables
    time_var = f.createVariable('time', 'f8', ('time',))
    plev_var = f.createVariable('plev', 'f8', ('plev',))
    lat_var = f.createVariable('lat', 'f8', ('lat',))
    for var, dict_ in ((time_var, time_attrs), (plev_var, plev_attrs), (lat_var, lat_attrs)):
        for key, value in dict_.items():
            setattr(var, key, value)

    # Make data Variables
    ehf_var = f.createVariable('ehf', 'f8', ('time', 'plev', 'lat',))
    emf_var = f.createVariable('emf', 'f8', ('time', 'plev', 'lat',))
    for var, dict_ in (
        (emf_var, {'long_name': 'eddy momentum flux', 'units': 'm^2 / s^2'}),
        (ehf_var, {'long_name': 'eddy heat flux', 'units': 'K m / s'}),
    ):
        for key, value in dict_.items():
            setattr(var, key, value)

    # Write numpy arrays to Variables
    time_var[:] = time
    plev_var[:] = plev
    lat_var[:] = lat
    ehf_var[:] = ehf
    emf_var[:] = emf

<ipython-input-96-ef9677b9da59>:27: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  time_attrs = {attr: time_var.getncattr(attr) for attr in time_var.ncattrs()}
<ipython-input-96-ef9677b9da59>:28: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  plev_attrs = {attr: plev_var.getncattr(attr) for attr in plev_var.ncattrs()}
<ipython-input-96-ef9677b9da59>:29: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  lat_attrs = {attr: lat_var.getncattr(attr) for attr in lat_var.ncattrs()}


In [19]:
%%bash
ncdump -h data/atmosphere_netcdf4.nc

netcdf atmosphere_netcdf4 {
dimensions:
	time = UNLIMITED ; // (200 currently)
	plev = 60 ;
	lat = 6 ;
variables:
	double time(time) ;
		time:long_name = "time" ;
		time:calendar = "360_day" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
	double plev(plev) ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:axis = "Z" ;
		plev:standard_name = "pressure level" ;
	double lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
		lat:standard_name = "latitude" ;
	double ehf(time, plev, lat) ;
		ehf:long_name = "eddy heat flux" ;
		ehf:units = "K*m/s" ;
	double emf(time, plev, lat) ;
		emf:long_name = "eddy momentum flux" ;
		emf:units = "m**2/s**2" ;
}


## xarray 

* The more "high-level" tool (requires fewer lines of code).
* The "new kid" on the block but *extremely* powerful.
* Includes very fancy `Dataset` and `DataArray` classes representing NetCDF files and NetCDF variables, respectively.
* Classes are easy-to-use and include hundreds of useful methods.
* The `DataArray` class acts as a sort-of wrapper on `numpy` arrays.
* Xarray can interpret [CF conventions](https://cfconventions.org), including [automatically applying](http://xarray.pydata.org/en/stable/generated/xarray.open_dataset.html) `add_offset`, `scale_factor`, and `_FillValue`.

There are also several other packages that integrate nicely with xarray:

* [dask](http://xarray.pydata.org/en/stable/user-guide/dask.html) for massive speedups on supercomputers by running math operations on "chunks" of your `DataArray`s in parallel. Useful when working with >10GB files.
* [metpy](https://unidata.github.io/MetPy/latest/) for meteorological analysis.
* [xgcm](https://github.com/xgcm/xgcm) and [xesmf](https://xesmf.readthedocs.io) for CDO-like horizontal and vertical grid manipulation inside python.
* [pint](http://pint.readthedocs.io) and [pint-xarray](https://pint-xarray.readthedocs.io/en/latest) for working with units and unit conversions (compatible with `DataArray`s).
* [cf-xarray](https://cf-xarray.readthedocs.io) for interpreting CF-standard attributes like `standard_name` and `axis`.
* [cftime](http://xarray.pydata.org/en/stable/user-guide/weather-climate.html#cftimeindex) for decoding non-Gregorian calendar dates in the `time` coordinate (e.g., `360_day` or `365_day`, often used by climate models).

Xarray will make your life much, much easier. I recommend using xarray most of the time.

### Basics

The `Dataset` looks like this (note the more useful, fancier formatting).

In [76]:
import xarray as xr
data = xr.open_dataset('data/atmosphere.nc')
data

<xarray.Dataset>
Dimensions:  (lat: 6, lon: 12, plev: 60, time: 200)
Coordinates:
  * time     (time) object 0000-01-01 06:00:00 ... 0000-04-10 18:00:00
  * plev     (plev) float32 8.444 25.33 42.22 59.11 ... 971.0 987.9 1.005e+03
  * lat      (lat) float32 -75.0 -45.0 -15.0 15.0 45.0 75.0
  * lon      (lon) float32 -165.0 -135.0 -105.0 -75.0 ... 75.0 105.0 135.0 165.0
Data variables:
    u        (time, plev, lat, lon) float64 ...
    v        (time, plev, lat, lon) float64 ...
    t        (time, plev, lat, lon) float64 ...

A `DataArray` is accessed by indexing the `Dataset` like a dictionary or using dot notation.

In [88]:
data['t']
data.t  # equivalent

<xarray.DataArray 't' (time: 200, plev: 60, lat: 6, lon: 12)>
array([[[[205.506622, ..., 205.506622],
         ...,
         [205.506622, ..., 205.506622]],

        ...,

        [[265.288391, ..., 265.288391],
         ...,
         [265.288391, ..., 265.288391]]],


       ...,


       [[[205.506622, ..., 205.506622],
         ...,
         [205.506622, ..., 205.506622]],

        ...,

        [[265.288391, ..., 265.288391],
         ...,
         [265.288391, ..., 265.288391]]]])
Coordinates:
  * time     (time) object 0000-01-01 06:00:00 ... 0000-04-10 18:00:00
  * plev     (plev) float32 8.444 25.33 42.22 59.11 ... 971.0 987.9 1.005e+03
  * lat      (lat) float32 -75.0 -45.0 -15.0 15.0 45.0 75.0
  * lon      (lon) float32 -165.0 -135.0 -105.0 -75.0 ... 75.0 105.0 135.0 165.0
Attributes:
    long_name:  temperature
    units:      K

Math is done directly on the `DataArray` rather than converting it to numpy arrays. This makes it easier to track attributes, coordinates, and dimensions across operations.

In [92]:
data * 5  # multiply all non-coordinate DataArrays by 5
data.t * 5  # multiply a single DataArray by 5

<xarray.DataArray 't' (time: 200, plev: 60, lat: 6, lon: 12)>
array([[[[1027.53311157, 1027.53311157, 1027.53311157, ...,
          1027.53311157, 1027.53311157, 1027.53311157],
         [1127.53311157, 1127.53311157, 1127.53311157, ...,
          1127.53311157, 1127.53311157, 1127.53311157],
         [1227.53311157, 1227.53311157, 1227.53311157, ...,
          1227.53311157, 1227.53311157, 1227.53311157],
         [1227.53311157, 1227.53311157, 1227.53311157, ...,
          1227.53311157, 1227.53311157, 1227.53311157],
         [1127.53311157, 1127.53311157, 1127.53311157, ...,
          1127.53311157, 1127.53311157, 1127.53311157],
         [1027.53311157, 1027.53311157, 1027.53311157, ...,
          1027.53311157, 1027.53311157, 1027.53311157]],

        [[1032.59933472, 1032.59933472, 1032.59933472, ...,
          1032.59933472, 1032.59933472, 1032.59933472],
         [1132.59933472, 1132.59933472, 1132.59933472, ...,
          1132.59933472, 1132.59933472, 1132.59933472],
         [1232.59933472, 1232.59933472, 1232.59933472, ...,
          1232.59933472, 1232.59933472, 1232.59933472],
         [1232.59933472, 1232.59933472, 1232.59933472, ...,
...
          1521.37557983, 1521.37557983, 1521.37557983],
         [1521.37557983, 1521.37557983, 1521.37557983, ...,
          1521.37557983, 1521.37557983, 1521.37557983],
         [1421.37557983, 1421.37557983, 1421.37557983, ...,
          1421.37557983, 1421.37557983, 1421.37557983],
         [1321.37557983, 1321.37557983, 1321.37557983, ...,
          1321.37557983, 1321.37557983, 1321.37557983]],

        [[1326.44195557, 1326.44195557, 1326.44195557, ...,
          1326.44195557, 1326.44195557, 1326.44195557],
         [1426.44195557, 1426.44195557, 1426.44195557, ...,
          1426.44195557, 1426.44195557, 1426.44195557],
         [1526.44195557, 1526.44195557, 1526.44195557, ...,
          1526.44195557, 1526.44195557, 1526.44195557],
         [1526.44195557, 1526.44195557, 1526.44195557, ...,
          1526.44195557, 1526.44195557, 1526.44195557],
         [1426.44195557, 1426.44195557, 1426.44195557, ...,
          1426.44195557, 1426.44195557, 1426.44195557],
         [1326.44195557, 1326.44195557, 1326.44195557, ...,
          1326.44195557, 1326.44195557, 1326.44195557]]]])
Coordinates:
  * time     (time) object 0000-01-01 06:00:00 ... 0000-04-10 18:00:00
  * plev     (plev) float32 8.444 25.33 42.22 59.11 ... 971.0 987.9 1.005e+03
  * lat      (lat) float32 -75.0 -45.0 -15.0 15.0 45.0 75.0
  * lon      (lon) float32 -165.0 -135.0 -105.0 -75.0 ... 75.0 105.0 135.0 165.0

In [93]:
data.mean(dim='time')  # average all non-coordinate DataArrays in time
data.t.mean(dim='time')  # average a single DataArray in time

<xarray.DataArray 't' (plev: 60, lat: 6, lon: 12)>
array([[[205.50662231, 205.50662231, 205.50662231, ..., 205.50662231,
         205.50662231, 205.50662231],
        [225.50662231, 225.50662231, 225.50662231, ..., 225.50662231,
         225.50662231, 225.50662231],
        [245.50662231, 245.50662231, 245.50662231, ..., 245.50662231,
         245.50662231, 245.50662231],
        [245.50662231, 245.50662231, 245.50662231, ..., 245.50662231,
         245.50662231, 245.50662231],
        [225.50662231, 225.50662231, 225.50662231, ..., 225.50662231,
         225.50662231, 225.50662231],
        [205.50662231, 205.50662231, 205.50662231, ..., 205.50662231,
         205.50662231, 205.50662231]],

       [[206.51986694, 206.51986694, 206.51986694, ..., 206.51986694,
         206.51986694, 206.51986694],
        [226.51986694, 226.51986694, 226.51986694, ..., 226.51986694,
         226.51986694, 226.51986694],
        [246.51986694, 246.51986694, 246.51986694, ..., 246.51986694,
         246.51986694, 246.51986694],
        [246.51986694, 246.51986694, 246.51986694, ..., 246.51986694,
...
         304.27511597, 304.27511597],
        [304.27511597, 304.27511597, 304.27511597, ..., 304.27511597,
         304.27511597, 304.27511597],
        [284.27511597, 284.27511597, 284.27511597, ..., 284.27511597,
         284.27511597, 284.27511597],
        [264.27511597, 264.27511597, 264.27511597, ..., 264.27511597,
         264.27511597, 264.27511597]],

       [[265.28839111, 265.28839111, 265.28839111, ..., 265.28839111,
         265.28839111, 265.28839111],
        [285.28839111, 285.28839111, 285.28839111, ..., 285.28839111,
         285.28839111, 285.28839111],
        [305.28839111, 305.28839111, 305.28839111, ..., 305.28839111,
         305.28839111, 305.28839111],
        [305.28839111, 305.28839111, 305.28839111, ..., 305.28839111,
         305.28839111, 305.28839111],
        [285.28839111, 285.28839111, 285.28839111, ..., 285.28839111,
         285.28839111, 285.28839111],
        [265.28839111, 265.28839111, 265.28839111, ..., 265.28839111,
         265.28839111, 265.28839111]]])
Coordinates:
  * plev     (plev) float32 8.444 25.33 42.22 59.11 ... 971.0 987.9 1.005e+03
  * lat      (lat) float32 -75.0 -45.0 -15.0 15.0 45.0 75.0
  * lon      (lon) float32 -165.0 -135.0 -105.0 -75.0 ... 75.0 105.0 135.0 165.0

### Example

The following example reads XYZT temperature and wind data, then saves YZT "eddy heat flux" and "eddy momentum flux".

In [8]:
%%bash
ncdump -h data/atmosphere.nc

netcdf atmosphere {
dimensions:
	time = UNLIMITED ; // (200 currently)
	plev = 60 ;
	lat = 6 ;
	lon = 12 ;
variables:
	float time(time) ;
		time:long_name = "time" ;
		time:calendar = "360_day" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
	float plev(plev) ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:axis = "Z" ;
		plev:standard_name = "pressure level" ;
	float lat(lat) ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
		lat:standard_name = "latitude" ;
	float lon(lon) ;
		lon:long_name = "longitude" ;
		lon:units = "degE" ;
		lon:axis = "X" ;
		lon:standard_name = "longitude" ;
	double u(time, plev, lat, lon) ;
		u:long_name = "zonal wind" ;
		u:units = "m/s" ;
	double v(time, plev, lat, lon) ;
		v:long_name = "meridional wind" ;
		v:units = "m/s" ;
	double t(time, plev, lat, lon) ;
		t:long_name = "temperature" ;
		t:units = "K" ;
}


In [94]:
import os
import sys
import xarray as xr

# Open dataset
ds = xr.open_dataset('data/atmosphere.nc', decode_times=False)

# Next perform calculations
emf = ((ds.u - ds.u.mean('lon')) * (ds.v - ds.v.mean('lon'))).mean('lon')
ehf = ((ds.t - ds.t.mean('lon')) * (ds.v - ds.v.mean('lon'))).mean('lon')
emf.name = 'emf'
ehf.name = 'ehf'
emf.attrs = {'long_name': 'eddy momentum flux', 'units': 'm^2 / s^2'}
ehf.attrs = {'long_name': 'eddy heat flux', 'units': 'K m / s'}

# Save file
ds = xr.Dataset({'emf': emf, 'ehf': ehf})
ds.to_netcdf('data/atmosphere_xarray.nc', mode='w')

In [95]:
%%bash
ncdump -h data/atmosphere_xarray.nc

netcdf atmosphere_xarray {
dimensions:
	time = 200 ;
	plev = 60 ;
	lat = 6 ;
variables:
	float time(time) ;
		time:_FillValue = NaNf ;
		time:long_name = "time" ;
		time:calendar = "360_day" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:axis = "T" ;
		time:standard_name = "time" ;
	float plev(plev) ;
		plev:_FillValue = NaNf ;
		plev:long_name = "pressure level" ;
		plev:units = "hPa" ;
		plev:axis = "Z" ;
		plev:standard_name = "pressure level" ;
	float lat(lat) ;
		lat:_FillValue = NaNf ;
		lat:long_name = "latitude" ;
		lat:units = "degN" ;
		lat:axis = "Y" ;
		lat:standard_name = "latitude" ;
	double emf(time, plev, lat) ;
		emf:_FillValue = NaN ;
		emf:long_name = "eddy momentum flux" ;
		emf:units = "m^2 / s^2" ;
	double ehf(time, plev, lat) ;
		ehf:_FillValue = NaN ;
		ehf:long_name = "eddy heat flux" ;
		ehf:units = "K m / s" ;
}


# Other tools

## MATLAB

If possible, I recommend against using [MATLAB](https://www.mathworks.com/products/matlab.html).

MATLAB's syntax is convenient, but python has many useful tools that have no MATLAB equivalent (e.g., dask, xarray, jupyter, metpy). Python is also open source and has a massive community, which helps with debugging code and developing even more useful tools.

If you know MATLAB but don't know python, an REU internship is a great time to learn (that's how I did it!).

If you really need to use MATLAB (for example, you will be working from MATLAB scripts provided by your advisor), more information on MATLAB's NetCDF utilities can be found [here](https://www.mathworks.com/help/matlab/network-common-data-form.html).

## Julia

If possible, I also recommend against the [Julia language](https://julialang.org/) for the time being.

Julia's NetCDF utilities are not as advanced as python's (yet), and the community is relatively small. The Julia language is a really awesome idea, and I'm excited about it! But I don't think Julia has proved itself worthy enough for us to jump ship from python. A major reason against using Julia is that xarray (and, especially, **dask**) exist.

If you disagree (or your advisor disagrees), the two major NetCDF-processing tools in Julia are [NetCDF.jl](https://github.com/JuliaGeo/NetCDF.jl) (for MATLAB-style syntax) and [NCDatasets.jl](https://github.com/Alexander-Barth/NCDatasets.jl) (for python's xarray-style syntax).

## NCL

A final tool I didn't mention is [NCL](https://www.ncl.ucar.edu) (the NCAR Command
Language). This is one of my favorites!

-   MATLAB: Everything is an array.
-   Python: Everything is an object (or dictionary, depending on who you
    ask).
-   NCL: Everything is a NetCDF-formatted dataset. If you're a
    geoscientist, this paradigm is pretty awesome.

Sadly, in 2021 you should **avoid using NCL** for two reasons:

-   NCL [is being
    deprecated](https://www.ncl.ucar.edu/Document/Pivot_to_Python/september_2019_update.shtml)
    (Unidata developers are now focusing on python tools).
-   As "cool" as NCL is, it is very slow... among the slowest tools (see [these
    benchmark results](https://github.com/lukelbd/atmos-benchmarks)).
    
As with MATLAB, I recommend using it only if your advisor gives you a bunch of NCL code to work with.

The following example reads XYZT temperature and wind data, then saves YZT "eddy heat flux" and "eddy momentum flux".

In [ ]:
%%bash
cat fluxes.ncl
rm data/fluxes_output.nc 2>/dev/null
ncl fluxes.ncl
ncdump -h data/atmosphere.nc

The `fluxes.ncl` script contains the following:

```
; Sample NCL file
; This is a comment
f = addfile("data/fluxes_input.nc", "r")
o = addfile("data/fluxes_output.nc", "c")
t = f->t
v = f->v
tstar = t - conform(t, dim_avg_n(t, 3), (/0, 1, 2/))  ; zonal temperature anomaly
vstar = v - conform(v, dim_avg_n(v, 3), (/0, 1, 2/))  ; zonal meridional-wind anomaly
ehf = dim_avg_n(tstar * vstar, 3)  ; eddy heat flux
copy_VarCoords(t(:, :, :, 0), ehf)
ehf@long_name = "eddy heat flux"
ehf@units = "K*m/s"
o->ehf = ehf
```

Running `ncl fluxes.ncl` will show the following (will not work inside this notebook).
```
Variable: t
Type: double
Total Size: 6912000 bytes
            864000 values
Number of Dimensions: 4
Dimensions and sizes:	[time | 200] x [plev | 60] x [lat | 6] x [lon | 12]
Coordinates: 
            time: [0.25..99.75]
            plev: [8.44375..1004.806]
            lat: [-75..75]
            lon: [-165..165]
Number Of Attributes: 2
  long_name :	temperature
  units :	K

Variable: v
Type: double
Total Size: 6912000 bytes
            864000 values
Number of Dimensions: 4
Dimensions and sizes:	[time | 200] x [plev | 60] x [lat | 6] x [lon | 12]
Coordinates: 
            time: [0.25..99.75]
            plev: [8.44375..1004.806]
            lat: [-75..75]
            lon: [-165..165]
Number Of Attributes: 2
  long_name :	meridional wind
  units :	m/s

Variable: ehf
Type: double
Total Size: 576000 bytes
            72000 values
Number of Dimensions: 3
Dimensions and sizes:	[time | 200] x [plev | 60] x [lat | 6]
Coordinates: 
            time: [0.25..99.75]
            plev: [8.44375..1004.806]
            lat: [-75..75]
Number Of Attributes: 2
  units :	K*m/s
  long_name :	eddy heat flux

```

In [18]:
%%bash
# rm data/fluxes_output.nc 2>/dev/null
# ncl fluxes.ncl
ncdump -h data/fluxes_output.nc

netcdf fluxes_output {
dimensions:
	time = 200 ;
	plev = 60 ;
	lat = 18 ;
variables:
	double ehf(time, plev, lat) ;
		ehf:units = "K*m/s" ;
		ehf:long_name = "eddy heat flux" ;
	float time(time) ;
		time:standard_name = "time" ;
		time:axis = "T" ;
		time:units = "days since 00-01-01 00:00:00" ;
		time:calendar = "360_day" ;
		time:long_name = "time" ;
	float plev(plev) ;
		plev:standard_name = "pressure level" ;
		plev:axis = "Z" ;
		plev:units = "hPa" ;
		plev:long_name = "pressure level" ;
	float lat(lat) ;
		lat:standard_name = "latitude" ;
		lat:axis = "Y" ;
		lat:units = "degN" ;
		lat:long_name = "latitude" ;
}


## Panoply

[Panoply](https://www.giss.nasa.gov) is maintained by NASA. This isn't software for manipulating data, but for displaying data. It's extremely useful for quickly viewing the data in your NetCDF files! Try playing around with it when you next have the spare time.

There is a similar, older software for displaying NetCDF data called [ncview](http://meteora.ucsd.edu/~pierce/ncview_home_page.html), but I find panoply more versatile and easier to use.

![](figures/panoply_demo.jpg)